In [3]:
import mne
print(f"MNE-Python version: {mne.__version__}")

ModuleNotFoundError: No module named 'mne'

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import mne
from mne.filter import filter_data, notch_filter
from scipy.signal import welch

In [ ]:
# Configuración con una frecuencia de muestreo más alta
sample_rate = 500  # Aumentado a 500 Hz para permitir el filtrado gamma
np.random.seed(42)

def simulate_eeg(duration=10, n_channels=8):
    n_samples = int(duration * sample_rate)
    time = np.linspace(0, duration, n_samples)

    # Componentes cerebrales con más variabilidad
    signals = [
        0.5 * np.sin(2 * np.pi * 2 * time + np.random.rand()),    # Delta
        0.3 * np.sin(2 * np.pi * 6 * time + np.random.rand()),    # Theta
        0.2 * np.sin(2 * np.pi * 10 * time + np.random.rand()),   # Alpha
        0.1 * np.sin(2 * np.pi * 25 * time + np.random.rand()),   # Beta
        0.05 * np.sin(2 * np.pi * 40 * time + np.random.rand())   # Gamma
    ]

    clean_eeg = sum(signals)
    noise = 0.1 * np.random.randn(n_samples, n_channels)
    eeg_data = clean_eeg[:, np.newaxis] + noise

    info = mne.create_info(n_channels, sample_rate, ch_types='eeg')
    raw = mne.io.RawArray(eeg_data.T, info)
    return raw, time

def filter_eeg(raw):
    # Filtro notch solamente a 50 Hz (armónicos debajo de Nyquist)
    raw.notch_filter(np.array([50]))  # Sólo 50 Hz

    bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 13),
        'Beta': (13, 30),
        'Gamma': (30, 45)  # Limitado a 45 Hz para mantenerse debajo de Nyquist
    }

    filtered = {}
    for band, (l_freq, h_freq) in bands.items():
        filtered[band] = raw.copy().filter(
            l_freq, h_freq,
            method='fir',
            fir_window='hamming',
            phase='zero-double',
            fir_design='firwin2'
        )

    return filtered

def plot_results(raw, filtered, time):
    plt.figure(figsize=(15, 12))

    # Señal original
    plt.subplot(3, 1, 1)
    plt.plot(time[:500], raw.get_data()[0, :500], 'k')  # Solo el primer segundo
    plt.title('Señal EEG original (Canal 1 - primer segundo)')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud (µV)')

    # Densidad espectral de potencia
    plt.subplot(3, 1, 2)
    f, Pxx = welch(raw.get_data()[0], fs=sample_rate, nperseg=1024)
    plt.semilogy(f, Pxx, 'k')
    plt.axvline(x=50, color='r', linestyle='--', label='50 Hz')
    plt.title('Densidad espectral de potencia')
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Potencia (dB/Hz)')
    plt.xlim(0, sample_rate/2)
    plt.legend()

    # Señales filtradas (primer segundo)
    plt.subplot(3, 1, 3)
    colors = {'Delta':'b', 'Theta':'g', 'Alpha':'r', 'Beta':'c', 'Gamma':'m'}
    for band, color in colors.items():
        data = filtered[band].get_data()[0, :500]
        plt.plot(time[:500], data, color, label=band, alpha=0.7)
    plt.title('Componentes espectrales filtradas (primer segundo)')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud (µV)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Ejecución
raw, time = simulate_eeg(duration=5)  # Duración reducida para la demostración
filtered = filter_eeg(raw)
plot_results(raw, filtered, time)
